Code to generate samples and learn the values of $\lambda$ and $m2$ for various sample sizes.

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, JuMP, Ipopt, Plots

# Define the Phi4Model structure for 2D
struct Phi4Model2D
    lattice_size::Int  # Lattice size (NxN)
    lambda::Float64    # Quartic coupling constant
    m2 :: Float64      # Mass term squared
end

# Function to calculate the total energy (kinetic + potential) of a 2D field configuration
function calculate_energy_2d(phi::Matrix{T}, model::Phi4Model2D) where T<:Real
    N = size(phi, 1)
    kinetic_energy = 0.0
    potential_energy = 0.0
    
    # Loop through all the lattice points and calculate kinetic and potential terms
    for i in 1:N, j in 1:N
        # Kinetic term: sum over nearest neighbors (discrete Laplacian)
        i_right = mod1(i + 1, N)
        j_up = mod1(j + 1, N)
        kinetic_energy += 0.5 * ((phi[i, j] - phi[i_right, j])^2 + (phi[i, j] - phi[i, j_up])^2)
        
        # Potential term: phi^4 interaction + mass term
        potential_energy += 0.5 * model.m2 * phi[i, j]^2 + 0.25 * model.lambda * phi[i, j]^4
    end
    
    return kinetic_energy + potential_energy
end

# Define the Phi4TargetDensity structure for 2D
struct Phi4TargetDensity2D
    dim::Int
    model::Phi4Model2D
end

# Implement the log-density function for Phi4 2D
function LogDensityProblems.logdensity(p::Phi4TargetDensity2D, phi_vector::Vector{T}) where T<:Real
    # Reshape the vector back into a 2D matrix
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::Phi4TargetDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{Phi4TargetDensity2D}) = LogDensityProblems.LogDensityOrder{0}()

# Function to generate data for the Phi4 model in 2D using Hamiltonian Monte Carlo (HMC)
function generate_phi4_data_hmc_2d(model::Phi4Model2D, n_samples::Int, n_adapts::Int = 1000)
    lattice_size = model.lattice_size
    phi4_target = Phi4TargetDensity2D(lattice_size^2, model)

    # Define the initial 2D field configuration (NxN matrix)
    initial_phi = randn(lattice_size, lattice_size)

    # Define a Hamiltonian system
    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, phi4_target, ForwardDiff)

    # Define a leapfrog solver with an initial step size chosen heuristically
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)

    # Define an HMC sampler with Multinomial sampling and Generalized No-U-Turn
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    # Run the sampler to draw samples from the Phi4 model
    samples_vector, stats = sample(hamiltonian, kernel, vec(initial_phi), n_samples, adaptor, n_adapts; progress=false)

    # Reshape the samples back into 2D matrices
    samples = [reshape(sample, lattice_size, lattice_size) for sample in samples_vector]

    return samples
end

# Score matching objective function for 2D
function score_matching_objective_2d(lambda::T, m2::T, data::Vector{Matrix{Float64}}) where T<:Real
    N = size(data[1], 1)
    n_samples = length(data)
    obj = zero(T)

    for sample in 1:n_samples
        phi = data[sample]
        for i in 1:N, j in 1:N
            x_ij = phi[i, j]
            # Nearest neighbors with periodic boundary conditions
            x_left = phi[mod1(i - 1, N), j]
            x_right = phi[mod1(i + 1, N), j]
            x_down = phi[i, mod1(j - 1, N)]
            x_up = phi[i, mod1(j + 1, N)]
            # Corrected kinetic term gradient (discrete Laplacian)
            kinetic_term = -(4 * x_ij - x_left - x_right - x_up - x_down)

            # Score function s(x_ij) including full kinetic term
            score_ij = kinetic_term - (m2 * x_ij + lambda * x_ij^3)
            # Second derivative s'(x_ij)
            s_prime = -(m2 + 3 * lambda * x_ij^2 + 4)
            # Add to the objective
            obj += T(0.5) * score_ij^2 + s_prime
        end
    end

    return obj / n_samples
end

# Function to estimate the parameters λ and m² using score matching in 2D
function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)
    
    @variable(model, λ)
    @variable(model, m2)
    
    register(model, :score_matching_obj, 2, (λ, m2) -> score_matching_objective_2d(λ, m2, data), autodiff=true)
    
    @NLobjective(model, Min, score_matching_obj(λ, m2))
    
    optimize!(model)
    
    if !(termination_status(model) in [MOI.OPTIMAL, MOI.LOCALLY_SOLVED])
        @warn "Optimization did not converge to an optimal solution"
    end
    
    estimated_lambda = value(λ)
    estimated_m2 = value(m2)
    
    return estimated_lambda, estimated_m2
end

# Function to plot errors for λ and m² versus sample sizes
function plot_errors_vs_sample_size(sample_sizes::Vector{Int}, errors_lambda::Vector{Float64}, errors_m2::Vector{Float64})
    # Compute the 1/sqrt(n) reference line
    reference_errors = 1 ./ sqrt.(sample_sizes)
    
    # Scale the reference line to start from the first error point for λ and m²
    scaled_ref_lambda = reference_errors * (errors_lambda[1] / reference_errors[1])
    scaled_ref_m2 = reference_errors * (errors_m2[1] / reference_errors[1])

    # Plot errors in λ vs sample size
    p1 = plot(sample_sizes, errors_lambda, xlabel="Sample Size", ylabel="Error in λ", 
              title="Error in λ vs Sample Size", marker=:circle, legend=:topright, label="Error in λ", xscale=:log10, yscale=:log10,lw=2,palette = :RdBu_4)
    plot!(sample_sizes, scaled_ref_lambda, linestyle=:dash, color=:black, label="1/√n")

    # Plot errors in m² vs sample size
    p2 = plot(sample_sizes, errors_m2, xlabel="Sample Size", ylabel="Error in m²", 
              title="Error in m² vs Sample Size", marker=:circle, legend=:topright, label="Error in m²", xscale=:log10, yscale=:log10,lw = 2,palette = :RdBu_4)
    plot!(sample_sizes, scaled_ref_m2, linestyle=:dash, color=:black, label="1/√n")

    return p1, p2
end

# Main function

Random.seed!(12345)
lattice_size = 4
n_reps = 30
true_model   = Phi4Model2D(lattice_size, 0.5, 0.75)  # Use m²

# single sample‑size in this example
sample_sizes = [2^(i) for i in 10:20]
N = length(sample_sizes)

# storage: each row = one repetition, each column = one sample_size
errors_λ  = zeros(n_reps, N)
errors_m2 = zeros(n_reps, N)

for rep in 1:n_reps
    println("=== rep $rep of $n_reps ===")
    for (i, n_samples) in enumerate(sample_sizes)
        println("  processing n_samples = $n_samples …")
        samples = generate_phi4_data_hmc_2d(true_model, n_samples)
        λ_est, m2_est = estimate_parameters_score_matching_2d(samples)

        errors_λ[rep,  i] = abs(λ_est  - true_model.lambda)
        errors_m2[rep, i] = abs(m2_est - true_model.m2)
    end
end

# average across repetitions
mean_err_λ  = vec(mean(errors_λ,  dims=1))
mean_err_m2 = vec(mean(errors_m2, dims=1))

# now plot the **averaged** errors
p1, p2 = plot_errors_vs_sample_size(sample_sizes, mean_err_λ, mean_err_m2)

savefig(p1, "error_in_lambda_avg.png")
savefig(p2, "error_in_m2_avg.png")
display(p1)
display(p2)

In [ ]:
colors = palette(:RdBu_4)
# take the last element (the deep blue)
blue = colors[end]

In [ ]:
function plot_errors_vs_sample_size(sample_sizes::Vector{Int}, errors_lambda::Vector{Float64}, errors_m2::Vector{Float64})
    # Compute the 1/sqrt(n) reference line
    reference_errors = 1 ./ sqrt.(sample_sizes)
    
    # Scale the reference line to start from the first error point for λ and m²
    scaled_ref_lambda = reference_errors * (errors_lambda[1] / reference_errors[1])
    scaled_ref_m2 = reference_errors * (errors_m2[1] / reference_errors[1])

    # Plot errors in λ vs sample size
    p1 = plot(sample_sizes, errors_lambda, xlabel="Sample Size", ylabel="Error in λ", 
              title="Error in λ vs Sample Size", marker=:circle, legend=:topright, label="Error in λ", xscale=:log10, yscale=:log10,lw=2,palette = :RdBu_4)
    plot!(sample_sizes, scaled_ref_lambda, linestyle=:dash, color=:black, label="1/√n")

    # Plot errors in m² vs sample size
    p2 = plot(sample_sizes, errors_m2, xlabel="Sample Size", ylabel="Error in m²", 
              title="Error in m² vs Sample Size", marker=:circle, legend=:topright, label="Error in m²", xscale=:log10, yscale=:log10,lw = 2,color = blue)
    plot!(sample_sizes, scaled_ref_m2, linestyle=:dash, color=:black, label="1/√n")

    return p1, p2
end

In [ ]:
p1, p2 = plot_errors_vs_sample_size(sample_sizes, mean_err_λ, mean_err_m2)

savefig(p1, "error_in_lambda_avg.png")
savefig(p2, "error_in_m2_avg.png")
display(p1)
display(p2)